In [1]:
import os
import numpy as np
import neuroglancer
from skimage import io
import matplotlib
from pprint import pprint
from matplotlib import pyplot as plt
from tqdm import tqdm
import cv2

In [2]:
animal = 'DK52'
filename = 'elastix_result.tif'
INPUT = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/CH1/registered'
OUTPUT = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/CH1/registered'
filepath = os.path.join(INPUT, filename)
volume = io.imread(filepath)
print(f'Volume shape={volume.shape} dtype={volume.dtype}')

Volume shape=(540, 352, 640) dtype=int16


In [ ]:
os.makedirs(OUTPUT, exist_ok=True)
for i in tqdm(range(0, volume.shape[2], 1)):
    #plt.title('Z {}'.format(i))
    #plt.imshow(arr[:,:,i], cmap="seismic")
    #plt.show()
    img = volume[:,:,i]
    img = np.rot90(img, 1, axes=(1,0))
    img = np.flip(img, axis=1)
    filepath = os.path.join(OUTPUT, str(i).zfill(3) + '.tif')
    cv2.imwrite(filepath, img)

In [7]:
# The default atlas created by brainreg is isotropic 25um
vm2 = np.swapaxes(volume, 0, 2)
#vm2 = np.flip(vm2, axis=0)
#vm2 = volume
print(f'new volume shape={vm2.shape}')
dimensions = neuroglancer.CoordinateSpace(
    names=['x', 'y', 'z'],
    units='um',
    scales=[25,25,25])
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    s.dimensions = dimensions
    s.layout = '4panel'
    s.layers['Allen'] = neuroglancer.ImageLayer(
        source=neuroglancer.LocalVolume(
            data=vm2,
            dimensions=dimensions,
            voxel_offset=(0, 0, 0),
        ))

print(viewer)

new volume shape=(640, 352, 540)
http://127.0.0.1:40785/v/428859168637a6af67baf16285aa4a59358a977d/


ERROR:tornado.application:Uncaught exception GET /neuroglancer/npz/428859168637a6af67baf16285aa4a59358a977d.1921707f1327a1fdafa7ba17ff7b198904ab058f/4,4,4/64,64,64/128,88,128 (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:40785', method='GET', uri='/neuroglancer/npz/428859168637a6af67baf16285aa4a59358a977d.1921707f1327a1fdafa7ba17ff7b198904ab058f/4,4,4/64,64,64/128,88,128', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/tornado/web.py", line 1713, in _execute
    result = await result
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/neuroglancer/server.py", line 325, in get
    data, content_type = await asyncio.wrap_future(
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/neuroglancer/local_volume.py", line 178, in get_encoded_s

ERROR:tornado.application:Uncaught exception GET /neuroglancer/npz/428859168637a6af67baf16285aa4a59358a977d.1921707f1327a1fdafa7ba17ff7b198904ab058f/4,4,4/128,0,64/160,64,128 (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:40785', method='GET', uri='/neuroglancer/npz/428859168637a6af67baf16285aa4a59358a977d.1921707f1327a1fdafa7ba17ff7b198904ab058f/4,4,4/128,0,64/160,64,128', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/tornado/web.py", line 1713, in _execute
    result = await result
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/neuroglancer/server.py", line 325, in get
    data, content_type = await asyncio.wrap_future(
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/neuroglancer/local_volume.py", line 178, in get_encoded_s

    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
ERROR:tornado.application:Uncaught exception GET /neuroglancer/npz/428859168637a6af67baf16285aa4a59358a977d.1921707f1327a1fdafa7ba17ff7b198904ab058f/2,2,2/192,64,128/256,128,192 (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:40785', method='GET', uri='/neuroglancer/npz/428859168637a6af67baf16285aa4a59358a977d.1921707f1327a1fdafa7ba17ff7b198904ab058f/2,2,2/192,64,128/256,128,192', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/tornado/web.py", line 1713, in _execute
    result = await result
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/neuroglancer/server.py", line 325, in get
    data, content_type = await asyncio.wrap_future(
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/usr/local/share/pipeline/lib/python3

    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
ERROR:tornado.application:Uncaught exception GET /neuroglancer/npz/428859168637a6af67baf16285aa4a59358a977d.1921707f1327a1fdafa7ba17ff7b198904ab058f/2,2,2/128,128,128/192,176,192 (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:40785', method='GET', uri='/neuroglancer/npz/428859168637a6af67baf16285aa4a59358a977d.1921707f1327a1fdafa7ba17ff7b198904ab058f/2,2,2/128,128,128/192,176,192', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/tornado/web.py", line 1713, in _execute
    result = await result
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/neuroglancer/server.py", line 325, in get
    data, content_type = await asyncio.wrap_future(
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/usr/local/share/pipeline/lib/pytho

...

    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
ERROR:tornado.application:Uncaught exception GET /neuroglancer/npz/428859168637a6af67baf16285aa4a59358a977d.1921707f1327a1fdafa7ba17ff7b198904ab058f/2,2,2/128,128,128/192,176,192 (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:40785', method='GET', uri='/neuroglancer/npz/428859168637a6af67baf16285aa4a59358a977d.1921707f1327a1fdafa7ba17ff7b198904ab058f/2,2,2/128,128,128/192,176,192', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/tornado/web.py", line 1713, in _execute
    result = await result
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/neuroglancer/server.py", line 325, in get
    data, content_type = await asyncio.wrap_future(
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/usr/local/share/pipeline/lib/pytho

    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
ERROR:tornado.application:Uncaught exception GET /neuroglancer/npz/428859168637a6af67baf16285aa4a59358a977d.1921707f1327a1fdafa7ba17ff7b198904ab058f/2,2,2/64,64,128/128,128,192 (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:40785', method='GET', uri='/neuroglancer/npz/428859168637a6af67baf16285aa4a59358a977d.1921707f1327a1fdafa7ba17ff7b198904ab058f/2,2,2/64,64,128/128,128,192', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/tornado/web.py", line 1713, in _execute
    result = await result
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/neuroglancer/server.py", line 325, in get
    data, content_type = await asyncio.wrap_future(
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/usr/local/share/pipeline/lib/python3.1

  File "/usr/local/share/pipeline/lib/python3.10/site-packages/neuroglancer/local_volume.py", line 178, in get_encoded_subvolume
    subvol = downsample.downsample_with_averaging(subvol, downsample_factor)
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/neuroglancer/downsample.py", line 30, in downsample_with_averaging
    counts = np.zeros(output_shape, np.int)
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/numpy/__init__.py", line 305, in __getattr__
    raise AttributeError(__former_attrs__[attr])
AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in Num

  File "/usr/local/share/pipeline/lib/python3.10/site-packages/neuroglancer/local_volume.py", line 178, in get_encoded_subvolume
    subvol = downsample.downsample_with_averaging(subvol, downsample_factor)
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/neuroglancer/downsample.py", line 30, in downsample_with_averaging
    counts = np.zeros(output_shape, np.int)
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/numpy/__init__.py", line 305, in __getattr__
    raise AttributeError(__former_attrs__[attr])
AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in Num

    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
ERROR:tornado.application:Uncaught exception GET /neuroglancer/npz/428859168637a6af67baf16285aa4a59358a977d.1921707f1327a1fdafa7ba17ff7b198904ab058f/2,2,2/0,0,128/64,64,192 (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:40785', method='GET', uri='/neuroglancer/npz/428859168637a6af67baf16285aa4a59358a977d.1921707f1327a1fdafa7ba17ff7b198904ab058f/2,2,2/0,0,128/64,64,192', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/tornado/web.py", line 1713, in _execute
    result = await result
  File "/usr/local/share/pipeline/lib/python3.10/site-packages/neuroglancer/server.py", line 325, in get
    data, content_type = await asyncio.wrap_future(
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/usr/local/share/pipeline/lib/python3.10/site-p